In [1]:
import os
import pandas as pd
import numpy as np
from optbinning import OptimalBinning

/home/huangdan/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/huangdan/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


(CVXPY) Feb 08 10:41:19 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Feb 08 10:41:19 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')


In [2]:
file_path = "/hdd1/sams/hcdr/dataset"
with open(file_path, 'r') as f:
    lines = f.readlines()

In [3]:
flag = False
rows = []
cnt = 0
sub = "'Spouse, partner'"
replaced_sub = "spouse_partner"

for idx, line in enumerate(lines):
    if line.strip() == "@DATA":
        flag = True
        continue

    if not flag:
        continue
    
    line = line.replace(sub, replaced_sub)
    cnt += 1
    row = [ i  for i in line.strip().split(',')]
    if len(row)!=70:
        print(row)
        break
    rows.append(row)
print(cnt)

244280


In [4]:
df = pd.DataFrame(rows)

In [5]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,0,135000.0,445500.0,20772.0,445500.0,0.026392,-10463,-235,-86.0,-3122,...,0,1,0.0,1,Y,'Commercial associate',Married,0,0,0
1,0,225000.0,675000.0,28597.5,675000.0,0.028663,-14872,-2488,-4213.0,-4208,...,0,1,0.0,0,N,Working,'Single / not married',0,0,0
2,1,76500.0,450000.0,22018.5,450000.0,0.0066709999999999,-15270,-223,-4754.0,-3529,...,0,1,0.0,0,N,Working,Separated,0,0,0
3,0,225000.0,1125000.0,36292.5,1125000.0,0.04622,-16645,-2541,-2429.0,-192,...,0,1,0.0,0,Y,'State servant',Married,0,0,0
4,0,117000.0,161730.0,8901.0,135000.0,0.01885,-15720,-7454,-1198.0,-4322,...,0,1,0.0,0,N,Working,Married,0,0,0


In [6]:
bin_cols = []
print(f"#col: {len(df.columns)}")
for col in df.columns:
    if col > 21:
        continue
    n = len(df[col].unique())
    if n > 24:
        print(f"{col}:{n}")
        bin_cols.append(col)

#col: 70
1:2022
2:5137
3:12964
4:801
5:81
6:17362
7:12263
8:15369
9:6148
12:107787
13:814
14:33
16:32
18:3736


In [7]:
# transfer these bin cols to float
for col in bin_cols:
    try:
        df[col] = df[col].astype(float)
    except:
        print(col)

In [8]:
df[69] = df[69].astype(int)

In [9]:
Y = df[69]
del df[69]
# X = df[range(69)]
X = df

In [10]:
def optbinning_data(df:pd.DataFrame, column:int, y:np.array):
    x = df[column].values

    optb = OptimalBinning(name="23", dtype="numerical", solver="cp",
                      monotonic_trend="auto", 
                      max_n_prebins=100,
                      min_prebin_size=0.001, time_limit=200)
    optb.fit(x,y)
    res = optb.transform(x, metric="indices")
    bin_number = max(res)
    status = optb.status
    set_n = len(df[column].unique())
    print(f"bin column:{column}, set size {set_n}, bin status {status}, bin_number {bin_number}")
    return res

In [11]:
for col in bin_cols:
    v = optbinning_data(X, col, Y.values)
    X[col] = v 

bin column:1, set size 2022, bin status FEASIBLE, bin_number 17
bin column:2, set size 5137, bin status FEASIBLE, bin_number 16
bin column:3, set size 12964, bin status OPTIMAL, bin_number 5
bin column:4, set size 801, bin status FEASIBLE, bin_number 21
bin column:5, set size 81, bin status OPTIMAL, bin_number 10
bin column:6, set size 17362, bin status OPTIMAL, bin_number 13
bin column:7, set size 12263, bin status FEASIBLE, bin_number 10
bin column:8, set size 15369, bin status OPTIMAL, bin_number 15
bin column:9, set size 6148, bin status FEASIBLE, bin_number 13
bin column:12, set size 107787, bin status OPTIMAL, bin_number 23
bin column:13, set size 814, bin status FEASIBLE, bin_number 31
bin column:14, set size 33, bin status OPTIMAL, bin_number 8
bin column:16, set size 32, bin status OPTIMAL, bin_number 9
bin column:18, set size 3736, bin status FEASIBLE, bin_number 13


In [19]:
# tokenization
col_to_value2token = {}
cnt = 0
for col in X.columns:
    col_values = X[col].values
    col_to_value2token[col] = {}
    col_values_set = set(col_values)
    for val in col_values_set:
        if val not in col_to_value2token[col]:
            col_to_value2token[col][val] = cnt
            cnt += 1
print(cnt)

544


In [21]:
# replace value with token
for col in X.columns:
    value_to_token = col_to_value2token[col]
    X[col] = X[col].map(value_to_token)

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_tmp, Y_train, Y_tmp = train_test_split(X,Y, test_size=0.2, random_state=2024)
X_val, X_test,Y_val, Y_test = train_test_split(X_tmp, Y_tmp, test_size=0.5, random_state=2024)

In [23]:
# save to libsvm
def save_dataset(X, Y, file_path:str):
    with open(file_path, 'w', encoding='utf-8') as f:
        n = len(X)
        for i in range(n):
            row = X.iloc[i].values 
            y = Y.iloc[i]
            items =[str(y)] + [str(i)+":1" for i in row]
            line = ' '.join(items) + '\n'
            f.write(line)

In [24]:
train_file = '/hdd1/sams/data/hcdr/train.libsvm'
val_file = '/hdd1/sams/data/hcdr/val.libsvm'
test_file = '/hdd1/sams/data/hcdr/test.libsvm'
save_dataset(X_train, Y_train, train_file)
save_dataset(X_val, Y_val, val_file)
save_dataset(X_test, Y_test, test_file)

In [27]:
print(f"train: POS/ALL -> {sum(Y_train.values.reshape(-1))}/{len(Y_train)}")
print(f"val: POS/ALL -> {sum(Y_val.values.reshape(-1))}/{len(Y_val)}")
print(f"test: POS/ALL -> {sum(Y_test.values.reshape(-1))}/{len(Y_test)}")
print(f"columns:{len(X_train.columns)}")

train: POS/ALL -> 15246/195424
val: POS/ALL -> 1913/24428
test: POS/ALL -> 1873/24428
columns:69


In [26]:
print(len(df))

244280
